In [1]:
import os
import pandas as pd
import grant_query
import publication_query
import utils
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)

In [2]:
KG_PATH = "../kg/data"
DERIVED_DATA_PATH = "../derived_data"

In [3]:
primary_authors = pd.read_csv(os.path.join(DERIVED_DATA_PATH, "primary_authors.csv"), dtype=str, keep_default_na=False)
primary_authors.head()

,authorId,author,aliases,projectSerialNum,doi
0,48292006,Allen C,"C Allen, Carl E Allen, Carl Allen, Carl E Alle...",HD105593,doi:10.3390/jcm12175435
1,3473746,Annapragada A,"Ananth V Annapragada, A Annapragada, A V Annap...",HD105593,doi:10.3390/jcm12175435
2,3473746,Annapragada A,"Ananth V Annapragada, A Annapragada, A V Annap...",HD105593,doi:10.1097/INF.0000000000003888
3,3473746,Annapragada A,"Ananth V Annapragada, A Annapragada, A V Annap...",HD105593,doi:10.1038/s41390-022-02108-6
4,3473746,Annapragada A,"Ananth V Annapragada, A Annapragada, A V Annap...",HD105593,doi:10.1016/j.jbi.2021.103818


In [4]:
id_to_author_id = pd.read_csv(os.path.join(DERIVED_DATA_PATH, "id_to_author_id.csv"), dtype=str, keep_default_na=False)

In [5]:
id_to_author_id.head()

,id,authorId,orcid,name,fullName,firstName,middleName,lastName
0,profileid:2563052,48292006,orcid:0000-0002-6625-739X,Allen CE,Carl E Allen,Carl,E,Allen
1,profileid:7039414,3473746,orcid:0000-0002-3156-9617,Annapragada AV,Ananth V Annapragada,Ananth,V,Annapragada
2,profileid:10320851,10999669,orcid:0000-0001-6532-8478,Bassiri H,Hamid Bassiri,Hamid,,Bassiri
3,profileid:7989301,6298375,orcid:0000-0001-5679-1217,Burns JC,Jane C Burns,Jane,C,Burns
4,profileid:8667619,2142194904,orcid:0000-0003-2915-2094,Chiu CY,Charles Yen Chiu,Charles,Yen,Chiu


In [6]:
primary_authors = primary_authors.merge(id_to_author_id, on="authorId", how="left")
primary_authors.fillna("")
primary_authors.head()

,authorId,author,aliases,projectSerialNum,doi,id,orcid,name,fullName,firstName,middleName,lastName
0,48292006,Allen C,"C Allen, Carl E Allen, Carl Allen, Carl E Alle...",HD105593,doi:10.3390/jcm12175435,profileid:2563052,orcid:0000-0002-6625-739X,Allen CE,Carl E Allen,Carl,E,Allen
1,3473746,Annapragada A,"Ananth V Annapragada, A Annapragada, A V Annap...",HD105593,doi:10.3390/jcm12175435,profileid:7039414,orcid:0000-0002-3156-9617,Annapragada AV,Ananth V Annapragada,Ananth,V,Annapragada
2,3473746,Annapragada A,"Ananth V Annapragada, A Annapragada, A V Annap...",HD105593,doi:10.1097/INF.0000000000003888,profileid:7039414,orcid:0000-0002-3156-9617,Annapragada AV,Ananth V Annapragada,Ananth,V,Annapragada
3,3473746,Annapragada A,"Ananth V Annapragada, A Annapragada, A V Annap...",HD105593,doi:10.1038/s41390-022-02108-6,profileid:7039414,orcid:0000-0002-3156-9617,Annapragada AV,Ananth V Annapragada,Ananth,V,Annapragada
4,3473746,Annapragada A,"Ananth V Annapragada, A Annapragada, A V Annap...",HD105593,doi:10.1016/j.jbi.2021.103818,profileid:7039414,orcid:0000-0002-3156-9617,Annapragada AV,Ananth V Annapragada,Ananth,V,Annapragada


## Create Researcher-AUTHORED-Publication nodes

In [7]:
author_map = {"id": "from", "doi": "to"}
authored = utils.rename_and_reorder_columns(primary_authors, author_map)
authored = authored[(authored["from"] != "") & (authored["to"] != "")]
authored.drop_duplicates(inplace=True)
authored = authored[authored["from"] != "s2authorid:None"]
authored.to_csv(os.path.join(KG_PATH, "relationships", "Researcher-AUTHORED-Publication_investigators.csv"), index=False)
print(f"Number of other Researcher nodes: {authored.shape[0]}")
authored.head()

Number of other Researcher nodes: 293


,from,to
0,profileid:2563052,doi:10.3390/jcm12175435
1,profileid:7039414,doi:10.3390/jcm12175435
2,profileid:7039414,doi:10.1097/INF.0000000000003888
3,profileid:7039414,doi:10.1038/s41390-022-02108-6
4,profileid:7039414,doi:10.1016/j.jbi.2021.103818


In [8]:
primary_authors_other = pd.read_csv(os.path.join("../derived_data/", "primary_authors_other.csv"), dtype=str, keep_default_na=False)
primary_authors_other["id"] = "s2authorid:" + primary_authors_other["authorId"]
authored_other = utils.rename_and_reorder_columns(primary_authors_other, author_map)
authored_other.drop_duplicates(inplace=True)
authored_other = authored_other[authored_other["from"] != "s2authorid:None"]
authored_other.to_csv(os.path.join(KG_PATH, "relationships", "Researcher-AUTHORED-Publication_other.csv"), index=False)
print(f"Number of other Researcher nodes: {authored_other.shape[0]}")
authored_other.head()

Number of other Researcher nodes: 3180


,from,to
1,s2authorid:80827919,doi:10.1016/j.cell.2021.05.002
2,s2authorid:2186155180,doi:10.1038/s41467-022-30357-w
3,s2authorid:2186155180,doi:10.1016/S2589-7500(22)00149-2
4,s2authorid:6865020,doi:10.1038/s41592-022-01444-z
5,s2authorid:2106780379,doi:10.3390/diagnostics13040707
